<html>
<head>
	<title></title>
</head>
<body>
<h1 style="text-align: center;line-height: 120%;">ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ВЫСШЕГО ОБРАЗОВАНИЯ НАЦИОНАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ УНИВЕРСИТЕТ «МОСКОВСКИЙ ЭНЕРГЕТИЧЕСКИЙ ИНСТИТУТ» Кафедра «РЗиАЭ»</h1>
<h2 style="text-align: center;line-height: 20%;">Лабораторная работа №3</h2>

<h3 style="text-align: center;line-height: 80%;"><em>Симплекс-метод</em></h3>

<p style="text-align: right;line-height: 40%;">Выполнил: студент группы Э-13м-19</p>
<p style="text-align: right;line-height: 40%;">Максимов Р.С.</p>

<p style="text-align: right;line-height: 40%;">Принял: Грачева Н. П.</p>
<h4 style="text-align: center;line-height: 100%;"><em> Москва, 2020</em></h3>
</body>
</html>

In [13]:
import numpy as np
import time
import random 
import scipy
from scipy import spatial
import math

Для реализации трех групп товаров коммерческое предприятие располагает тремя видами ограниченных материально-денежных ресурсов в количестве b1, b2, b3, единиц. При этом для продажи 1 группы товаров на 1 тыс. руб. товарооборота расходуется ресурса первого вида в количестве a11 единиц, ресурса второго вида в количестве a21 единиц, ресурса третьего вида в количестве a31 единиц. Для продажи 2 и 3 групп товаров на 1 тыс. руб. товарооборота расходуется соответственно ресурса первого вида в количестве a12, a13 единиц, ресурсов второго вида в количестве a22, a23 единиц, ресурсов третьего вида в количестве a32, a33 единиц. Прибыль от продажи трех групп товаров на 1 тыс. руб. товарооборота составляет соответственно c1, c2, c3 тыс. руб

Необходимо, определить плановый объем и структуру товарооборота так, чтобы прибыль торгового предприятия была максимальной.

In [82]:
def search_min(f):
    minimum = 9*max(f)
    for i in range(len(f)):
        if f[i] < minimum and not (f[i] == 0):
            minimum = f[i]
    return minimum
        
        

In [112]:
def simplex_method(a,b,F):
    b = np.array(b)
    F = np.array(F)
    F = F*-1
    a.extend([F])
    allTable = np.array(a)    
#     print(allTable) 
    y = np.eye(len(b),len(b)-1)
    b = np.hstack((b,[0]))
    b_column = b.reshape(len(b),1)
    y = np.eye(len(b),len(b)-1) #для добавления,формирования конечной матрицы
    g = np.hstack((allTable, y)) 
    end_table = np.hstack((g, b_column)) # матрица
    last_row = end_table.shape[0]-1
    last_colomn = end_table.shape[1]-1
    min_number_row = search_min(end_table[last_row,:])
#     print(min_number_row)
    i,  = np.where(end_table[last_row,:] == min_number_row)
#     print(type(i))
#     print(i)
    i = int(i)
    
    bases = np.arange(len(F)+1,last_colomn+1)
#     print(bases)
    iks = np.arange(1,len(F)+1)
    
    
    while min_number_row < 0:
#         print('Актуальная матрица = ', end_table)
        #находим индекс самого минимального и отрицательного элемента
        i,  = np.where(end_table[last_row,:] == min_number_row)
        i = int(i)
#         print("Индекс минимального элемента в последней строке = ", i)
        #нулевой разрешающий столбец
        column = np.zeros((len(b),1))
        #формируем разрешающий стобец
        for j in range(len(b)):
            column[j,0] = end_table[j,last_colomn] / end_table[j,i]
#         print("Дополнительный столбец = ", column)
        #находим минимальное число из разрешающего столбца
        min_number_column = search_min(column[:len(column)-1,0]) #все кроме последнего элемента(он ноль)
#         print("Мин. число из дополнительного столбца = ", min_number_column)
        #находим индекс, таким образом, находим разрешающуюю строку
        k,  = np.where(column[:,0] == min_number_column)
#         print("Индекс минимального элемента в дополнительном столбце = ", k)
        k = int(k)
        #Изменяем текущий базис
        bases[k] = i + 1
        #находим разрешающий коэф
        enable_coef = (end_table[k,i])
#         print('Разрешающий коэф = ',enable_coef)
        #делим разрещающую строку на разрешающий коэф.
        end_table[k,:] = end_table[k,:] / enable_coef
        #формируем нулевую разрешающую строку
        enable_row = end_table[k,:]
        for m in range(end_table.shape[0]):
            if not(m == k):
                end_table[m,:] =  end_table[m,:] - enable_row * end_table[m,i]
        min_number_row = min(end_table[last_row,:])
#     print(bases)
    print('Конечная матрица = ', end_table)
    #вывод значений, формирование базисов
    bases_X = np.arange(0,len(F))*0 #лист из нулевых базисов
    X = [0.]*len(F) #лист из нулевых иксов
    #
    for count in range(len(b)-1):       
        number_basis = bases[count]   #актуальный базис в цикле из списка всех конечных базисов     
        if number_basis in iks: 
            index_b, = np.where(bases == number_basis) #если номер базиса равен искомому номеру икса, то  
            index_b = int(index_b) #находим индекс
            X[number_basis-1] = float(end_table[index_b,last_colomn]) 

        
#     print(X)
    maxFunc = end_table[last_row,last_colomn]
#     print(maxFunc)
    return X,maxFunc

In [117]:
a = [[32, 11, 4],[5, 16, 32],[21, 12, 5]]
b = [520, 550, 90]
c = [5, 3, 7]
F = c
x,f = simplex_method(a,b,c)
print('Оптимальные X = ', x)
print('Значение функции при оптимальных X = ',f)

Конечная матрица =  [[ 0.00000000e+00 -5.74188563e+00  0.00000000e+00  1.00000000e+00
   1.17465224e-01 -1.55177743e+00  4.44945904e+02]
 [ 0.00000000e+00  4.26584235e-01  1.00000000e+00  0.00000000e+00
   3.24574961e-02 -7.72797527e-03  1.71561051e+01]
 [ 1.00000000e+00  4.69860896e-01  0.00000000e+00  0.00000000e+00
  -7.72797527e-03  4.94590417e-02  2.00927357e-01]
 [ 0.00000000e+00  2.33539413e+00  0.00000000e+00  0.00000000e+00
   1.88562597e-01  1.93199382e-01  1.21097372e+02]]
Оптимальные X =  [0.2009273570324575, 0.0, 17.15610510046368]
Значение функции при оптимальных X =  121.09737248840804


In [118]:
a = [[32, 11, 4, 2],[5, 16, 32,2],[21, 12, 5, 0],[15, 12, 13,10],[10, 9, 0,8]]
b = [520, 550, 90, 110, 300]
c = [5, 3, 7, 8]
x,f = simplex_method(a,b,c)
print('Оптимальные X = ', x)
print('Значение функции при оптимальных X = ',f)

Конечная матрица =  [[ 2.90e+01  8.60e+00  1.40e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00
  -2.00e-01  0.00e+00  4.98e+02]
 [ 2.00e+00  1.36e+01  2.94e+01  0.00e+00  0.00e+00  1.00e+00  0.00e+00
  -2.00e-01  0.00e+00  5.28e+02]
 [ 2.10e+01  1.20e+01  5.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00
   0.00e+00  0.00e+00  9.00e+01]
 [ 1.50e+00  1.20e+00  1.30e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00
   1.00e-01  0.00e+00  1.10e+01]
 [-2.00e+00 -6.00e-01 -1.04e+01  0.00e+00  0.00e+00  0.00e+00  0.00e+00
  -8.00e-01  1.00e+00  2.12e+02]
 [ 7.00e+00  6.60e+00  3.40e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   8.00e-01  0.00e+00  8.80e+01]]
Оптимальные X =  [0.0, 0.0, 0.0, 11.0]
Значение функции при оптимальных X =  88.0


C:\Users\maksimov\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in double_scalars
